<a href="https://colab.research.google.com/github/farhanrhine/deep-learning/blob/main/hyperparameter_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hyperparamter tuning of Neural Networks using KerasTuner


In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
url = 'https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv'
data = pd.read_csv(url)

In [4]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
data.shape

(768, 9)

In [7]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### WITHOUT KERAS TUNER

In [10]:
def build_baseline_model():
    model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

baseline_model = build_baseline_model()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
baseline_model = build_baseline_model()

In [12]:
baseline_history = baseline_model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 210ms/step - accuracy: 0.6628 - loss: 3.5721 - val_accuracy: 0.7073 - val_loss: 1.4723
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5975 - loss: 1.4918 - val_accuracy: 0.5935 - val_loss: 1.1455
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6879 - loss: 0.9733 - val_accuracy: 0.6179 - val_loss: 0.9959
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6844 - loss: 0.8917 - val_accuracy: 0.5935 - val_loss: 0.8544
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7039 - loss: 0.7618 - val_accuracy: 0.6179 - val_loss: 0.8067
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7127 - loss: 0.6385 - val_accuracy: 0.5935 - val_loss: 0.7686
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6465 - loss: 0.7526 - val_accuracy: 0.5854 - val_loss: 0.7611
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6584 - loss: 0.7221 - val_accuracy: 0.6016 - 

In [13]:
baseline_loss, baseline_accuracy = baseline_model.evaluate(X_test, y_test, verbose=0)
print(f'Baseline Model Accuracy: {baseline_accuracy:.4f}')
print(f'Baseline Model Loss: {baseline_loss:.4f}')

Baseline Model Accuracy: 0.6494
Baseline Model Loss: 0.7006


### WITH KERAS TUNER

In [14]:
import keras_tuner as kt

In [15]:
def model_builder(hp):
    model = Sequential()

# units = neuron
    hp_units = hp.Int('units', min_value=8, max_value=32, step=8)
    model.add(Dense(units=hp_units, activation='relu', input_shape=(X_train.shape[1],))) # input layer
    model.add(Dense(units=hp_units, activation='relu')) #hidden layer
    model.add(Dense(1, activation='sigmoid')) # output layer

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [16]:
tuner = kt.RandomSearch(model_builder,
                        objective='val_accuracy',
                        max_trials=5,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Clear the tuner directory to start a fresh search
#!rm -rf my_dir/intro_to_kt

In [18]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1) # train

Trial 5 Complete [00h 00m 33s]
val_accuracy: 0.6693766911824545

Best val_accuracy So Far: 0.7398374080657959
Total elapsed time: 00h 02m 54s


In [19]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [20]:
print(best_hps.values)


{'units': 8, 'learning_rate': 0.01}


In [21]:
tuned_model_kt = model_builder(best_hps)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
tuned_model_kt.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 461 (1.80 KB)

 Trainable params: 153 (612.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 308 (1.21 KB)

In [26]:
tuned_history_kt = tuned_model_kt.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7304 - loss: 0.5408 - val_accuracy: 0.7317 - val_loss: 0.5142
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7420 - loss: 0.5134 - val_accuracy: 0.7561 - val_loss: 0.5381
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7179 - loss: 0.5635 - val_accuracy: 0.6911 - val_loss: 0.5789
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7226 - loss: 0.5431 - val_accuracy: 0.6667 - val_loss: 0.6586
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6639 - loss: 0.6526 - val_accuracy: 0.6748 - val_loss: 0.5524
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7151 - loss: 0.5726 - val_accuracy: 0.6748 - val_loss: 0.5557
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7384 - loss: 0.5158 - val_accuracy: 0.7480 - val_loss: 0.5265
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7379 - loss: 0.5205 - val_accuracy: 0.6911 - val_loss:

In [27]:
tuned_loss_kt, tuned_accuracy_kt = tuned_model_kt.evaluate(X_test, y_test, verbose=0)

In [28]:
print(f'Tuned Model Accuracy (Keras Tuner): {tuned_accuracy_kt:.4f}')
print(f'Tuned Model Loss (Keras Tuner): {tuned_loss_kt:.4f}')

Tuned Model Accuracy (Keras Tuner): 0.6688
Tuned Model Loss (Keras Tuner): 0.6124
